# cool links
http://rna.tbi.univie.ac.at/forna/ -- for RNA molecule visualization

https://github.com/DasLab/draw_rna - a python library

https://academic.oup.com/nar/article/46/11/5381/4994207


https://www.tensorflow.org/lattice -- maybe try this out sometime?

#add the draw_rna library to this environment
! git clone https://github.com/DasLab/draw_rna.git #need to find a way to say yes
! python draw_rna/setup.py install

#add forna library
'''
! git clone https://github.com/ViennaRNA/forna.git
! cd forna/
! ls forna/'''

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import sklearn
import matplotlib.pyplot as plt

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
'''
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
'''
# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#these are the columns we have to predict, in same order as submission file format
target_cols = ['reactivity', 'deg_Mg_pH10', 'deg_pH10', 'deg_Mg_50C', 'deg_50C']

# Load Data

In [ ]:
def read_json(filename):
    '''
    reads in train/test json data as pandas DataFrame
    '''
    file = open(filename)
    df = pd.read_json(path_or_buf = file, orient = 'records', lines = True)
    return df

In [ ]:
#load in training set tabular data

train_df = read_json('../input/stanford-covid-vaccine/train.json')

print(train_df['id'].nunique())
print(train_df.columns)
train_df

In [ ]:
#load in test set tabular data

test_df = read_json('../input/stanford-covid-vaccine/test.json')


#see which feature columns are only in the training set & not the test set
print('Features only in training set (not including target columns):') 
set(train_df.columns) - set(test_df.columns) - set(target_cols)

In [ ]:
test_df

# Visualize

In [ ]:
'''! ls
#! ls draw_rna
! ls forna

! python forna/forna_server.py -s -d'''

In [ ]:
'''seq = train_df.loc[0, 'sequence']
struct = train_df.loc[0, 'structure']

seq, struct'''

# Preprocess and Feature Engineering

figure out way to encode structure

* try looking at how the **different target values are correlated** -- maybe we can use the non-scored ones to predict the scored ones?
    * take care to avoid leakage -- you won't have some target values at test time, but maybe we can have separate model to predict those columns beforehand?
    
    
* try integrating bpps matrices

In [ ]:
def unpack_df_lists(df, col_names):
    '''
    turn list-like elements of dataframe into tabular data
    
    works great
    '''
    if isinstance(col_names, str): #if string is passed in, convert to list for convenience
        col_names = [col_names]
    
    all_series = [df[c] for c in col_names] #select relevant columns
    unpacked = [ser.explode() for ser in all_series] #unpack lists for each feature series
    
    data = pd.concat(unpacked, axis = 1) #concat unpacked columns together
    
    #merge unpacked columns with original
    original = df.drop(col_names, axis = 1) #drop columns with list elements
    data = original.join(data) #then join unpacked data to original df
    
    return data

In [ ]:
#modify this to work with sequence models
'''
def feature_engineer(df, train = True, **kwargs):
    
    
    unpack_cols = ['reactivity_error', 'deg_error_Mg_pH10', 'deg_error_pH10',
       'deg_error_Mg_50C', 'deg_error_50C', 'reactivity', 'deg_Mg_pH10',
       'deg_pH10', 'deg_Mg_50C', 'deg_50C'] #only need to unpack things in training set
    
    #unpack list elements (only training set needs to be unpacked)
    if train:
        data = unpack_df_lists(df, unpack_cols)
    else: #if test data, need to add rows manually
        data = df.copy()
        data['temp'] = data.apply(lambda row: [0] * row['seq_length'], axis = 1) #adds temp column with list-like elements, of len(seq_scored) for that row 
        data = unpack_df_lists(data, 'temp') #unpack to right length using this function
        del data['temp'] #delete the temp column
        #this works great!
        
    #adds seqpos column to record position of each row in each id's individual sequence
    data['seqpos'] = 1
    data['seqpos'] = data.groupby('id').cumsum()['seqpos'] - 1
    
    #adds nucleotide column to record base (A,C,G,U) at position seqpos
    seq_temp = pd.concat([data['sequence'],data['seqpos']], axis = 1)
    data['nucleotide'] = seq_temp.apply(lambda row: row['sequence'][row['seqpos']], axis = 1) #get base at seqpos in sequence string
    
    #adds pred_loop_seqpos column to record predicted loop type at position seqpos
    loop_temp = pd.concat([data['predicted_loop_type'],data['seqpos']], axis = 1)
    data['pred_loop_seqpos'] = loop_temp.apply(lambda row: row['predicted_loop_type'][row['seqpos']], axis = 1) #get type at seqpos in predicted_loop_type string 
    
    #do one-hot encoding on nucleotide column? or label encoding?
    data = pd.get_dummies(data, columns = ['nucleotide','pred_loop_seqpos']) #do one-hot encoding on predicted_loop_type & nucleotide column
    
    return data
'''

feature engineering notes:

3.61 s ± 37 ms per loop (mean ± std. dev. of 7 runs, 1 loop each) -- with nucleotide column engineering

266 ms ± 7.71 ms per loop (mean ± std. dev. of 7 runs, 1 loop each) -- without

6.31 s ± 55 ms per loop (mean ± std. dev. of 7 runs, 1 loop each) -- seq_temp groupby





In [ ]:
#prob better to use tokenizer from tensorflow keras

tokenize_cols = ['sequence', 'structure', 'predicted_loop_type']
def tokenize_df(df, tokenizer, cols = tokenize_cols):
    '''
    tokenizer is a tensorflow keras Tokenizer that has been already fitted on text
    '''
    data = df.copy()
    for c in tokenize_cols:
        data[c] = tokenizer.texts_to_sequences(data[c])
    return data

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(filters = None, lower = False, char_level = True)
tokenizer.fit_on_texts('().ACGUBEHIMSX')

#filter and tokenize
temp = train_df[train_df['SN_filter'] == 1]
temp = tokenize_df(temp, tokenizer)

train_df = temp

train_df

In [ ]:

#load in bpp data as well

def read_bpp(ids):
    base_path = '../input/stanford-covid-vaccine/bpps/' #where all bpp files are
    filepaths = [f'{base_path}{rna_id}.npy' for rna_id in ids]
    bpps = [np.load(fp) for fp in filepaths]
    return np.stack(bpps, axis = 0) #might not staa

In [ ]:
#this cell is to load bpp train data

#get ids of training data to create files
ids = train_df['id']
bpp_train = read_bpp(ids)

bpp_train

# this is for tabular data model input -- sequence model doesn't use

#unpack the lists and add a seqpos column
#also convert dtypes for memory savings
unpack_cols = ['reactivity_error', 'deg_error_Mg_pH10', 'deg_error_pH10',
       'deg_error_Mg_50C', 'deg_error_50C', 'reactivity', 'deg_Mg_pH10',
       'deg_pH10', 'deg_Mg_50C', 'deg_50C'] #only need to unpack things in training set


temp = train_df[train_df['SN_filter'] == 1] #only select good quality examples to train on
temp = feature_engineer(temp)

#check proportions of nucleotides and loop types
for b in ['A','C','G','U']:
    print(b, temp['nucleotide_'+b].mean())
    
for b in ['S','M','I','B','H','E','X']:
    print(b, temp['pred_loop_seqpos_'+b].mean())


#now convert dtypes (should put this code in function)
print('train_df memory (MB):', train_df.memory_usage(deep = True).sum() * 1e-6)
print('temp_df memory before (MB):', temp.memory_usage(deep = True).sum() * 1e-6)

temp['SN_filter'] = temp['SN_filter'].astype('uint8')
temp['signal_to_noise'] = temp['signal_to_noise'].astype('float16') #seems like signal_to_noise isn't too precise
temp[['seq_length','seq_scored']] = temp[['seq_length','seq_scored']].astype('uint8') #seq_length and seq_scored should only be 107 or 130
temp[unpack_cols] = temp[unpack_cols].astype('float16') #looks like we don't need much precision to represent these cols -- may need to verify if model suffers
temp['seqpos'] = temp['seqpos'].astype('uint8') #max of uint8 is 255, which is fine -- seqpos is only ever 68 or 91

print('temp_df memory after (MB):', temp.memory_usage(deep = True).sum() * 1e-6)
print(temp.dtypes)


train_df = temp
train_df

* Idea: use signal_to_noise ratio column as example weight? seems like higher signal-to-noise means that that example is more useful; maybe something to do with Bayesian regression -- less confidence in low signal-to-noise, higher confidence in high signal-to-noise
    * look into "sample_weight" in tf keras model.fit or sklearn model.fit
    * Even if filtering, maybe can still use signal_to_noise as a sample weight -- surely high signal_to_noise is still better overall


"[Signal/noise is defined as mean( measurement value over 68 nts )/mean( statistical error in measurement value over 68 nts)]"

https://www.kaggle.com/isaienkov/openvaccine-eda-feature-engineering-modeling

* he splits the RNA structure into groups of 3 -- possibly good idea, given 3 nucleotides needed for 1 amino acid (may not be wise, has been mentioned before by organizer of competition)

# Target EDA


see here for more: https://www.kaggle.com/robikscube/openvaccine-covid-19-mrna-starter-eda

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

corr_data = train_df.drop(['index','id','sequence','structure','predicted_loop_type', 'seq_length','seq_scored'], axis = 1)
corr_data = unpack_df_lists(corr_data, ['reactivity_error', 'deg_error_Mg_pH10', 'deg_error_pH10',
       'deg_error_Mg_50C', 'deg_error_50C', 'reactivity', 'deg_Mg_pH10',
       'deg_pH10', 'deg_Mg_50C', 'deg_50C']).convert_dtypes()


print(corr_data)

corr_data = corr_data.corr()

corr_data

In [ ]:
mask = np.ma.masked_inside(corr_data.values, -0.15, 0.15).mask #get most powerful features

plt.figure(figsize = (8,8))
sns.heatmap(corr_data, annot = True, mask = mask)

EDA notes:

**With Filtering SN_filter == 1:**
* even higher correlation between targets! (0.6 - 0.8)


**W/O Filtering SN_filter == 1:**
* having a predicted loop type of S seems to be decently negatively correlated (-0.2) with some target values

* seqpos seems to not be strongly correlated to target values over all data -- however, for some individual RNA sequences, it seems that there is a correlation (see robikscube EDA)

* seems like there's a pretty good (~0.4 - 0.5) correlation between each of the target values, at least in train set
* maybe use one target column's values to predict the others?
* how to exploit this -- what if i predicted one target column at a time, then used that column to "bootstrap" off -- used the columns I predicted to predict another column
    * issue -- surely the ordering of which column is predicted first will greatly affect other predictions
    * idea -- could try different orderings of prediction order, then average out -- maybe try Shapley value?

# Define Metrics

In [ ]:
'''
For tensorflow compatibility, metrics should have signature f(y_true, y_pred)
For sklearn compatibility, metrics should have signature f(y_true, y_pred, **kwargs)
'''


def score(raw_values = False, use_tf = False, **kwargs):
    '''
    This is competition metric: Mean Columnwise Root Mean Square Error (MCRMSE)
    Averages RMSE loss over all scored columns (only 3 are scored)
    
    Parameters:
    For now, kwargs is ignored
    tf -- True if using in tensorflow, false if not
    col_dict is a dictionary that maps column number index to column name
        keys 'reactivity', 'deg_Mg_pH10', 'deg_Mg_50C'
        values are numeric index of that column in y_pred
    raw_values determines if losses for each column are returned or just the average
        if True, losses for each of columns are returned
        if False, only average is returned
    
    Returns a loss function that computes MCRMSE for scored columns
    '''    
    
    #these columns are scored
    col_dict = {
        'reactivity':0,
        'deg_Mg_pH10':1,
        'deg_Mg_50C':3
    }
    
    #these are the columns to not score
    unscored = set([0,1,2,3,4]) - set(col_dict.values())
    
    #choose output format of metric
    multi = 'uniform_average'
    if raw_values:
        multi = 'raw_values'
    
    def loss(y_true, y_pred):
        '''
        y_true & y_pred may have more columns than needed for scoring
        select only necessary ones for scoring

        y_true & y_pred have shapes (n, 5, len), where n is # of id_seqpos combos, len is length of sequence
        '''
        from sklearn.metrics import mean_squared_error
        y_true = np.array(y_true) #convert to np for convenience
        y_pred = np.array(y_pred)

        
        #only select scored columns
        y_true = y_true[:, list(col_dict.values())]
        y_pred = y_pred[:, list(col_dict.values())]
        

        #sqrt all outputs, then average them (if not raw_values)
        metric = mean_squared_error(y_true, y_pred, squared = False, multioutput = multi)
        return metric
    
    def loss_tf(y_true, y_pred):
        '''
        WIP -- need to prioritize scored columns
        
        '''
        
        import tensorflow as tf
        import tensorflow.keras.backend as tf_kb
        
        #print(type(y_true),type(y_pred))
        
        #note -- y_true may be in different format than y_pred
        #y_true = tf.convert_to_tensor(y_true)
        #y_pred = tf.convert_to_tensor(y_pred)
        
        
        #set values in y_pred to be same as y_true for unscored columns
        #this ensures that these columns do not contribute to loss
        '''for c in unscored:
            y_pred[:, c] = y_true[:, c]'''
            
        '''
        colwise_mse = tf_kb.mean(tf_kb.square(y_true - y_pred)) #first take average squared difference over row examples (shape 1x3)
        return tf_kb.mean(tf_kb.sqrt(colwise_mse)) #then sqrt and take average over columns (shape 1x1 / scalar)
        '''        
        
        
        colwise_mse = tf.reduce_mean(tf.square(y_true - y_pred), axis=1)
        return tf.reduce_mean(tf.sqrt(colwise_mse), axis=1)
        
        
    if not use_tf:
        return loss
    else:
        return loss_tf

# Define X, y

(no need to define sample weights anymore, since private LB is also filtered now)

try RNN models -- they seem to work good
https://www.kaggle.com/neithermannormachine/openvaccine-gru-lstm/edit

or maybe Conv1D?

https://github.com/tkipf/keras-gcn -- graph convolutions (may be good for nonlinear RNA sequence structure)
https://towardsdatascience.com/how-to-do-deep-learning-on-graphs-with-graph-convolutional-networks-7d2250723780

possibly regression chaining using target correlation


In [ ]:
#Define the training data
#create X_train and y_train

#drop for now when training, unless i can find a way to use them for feature engineering
train_only_cols = ['reactivity_error', 'deg_error_Mg_pH10', 'deg_error_pH10', 'deg_error_Mg_50C', 'deg_error_50C'] #features only in train set
signal_cols = ['signal_to_noise','SN_filter']
drop_cols = [
             'seq_length', 'seq_scored', #don't actually use seq_length and seq_scored for training - just metadata
            'index', 'id']  #also not actually useful for training
train_drop_cols = drop_cols + target_cols + train_only_cols + signal_cols


#X_train should be an iterable, with shape (n, 107, 3)
#each row should be a single np array
X_train = (train_df.drop(train_drop_cols, axis = 1) #selects only relevant columns
                    .apply(lambda row: [e for e in row], axis = 1) #concatenates each element in row to list
                    .apply(lambda e : np.array(e)) #creates 2d numpy array from those elements
          )
X_train = np.stack(X_train.values, axis = 0) #stacking might not work for test data, since there is two different shapes

y_train = (train_df[target_cols]
               .apply(lambda row: [e for e in row], axis = 1) #concatenates each element in row to list
                .apply(lambda e : np.array(e)) #creates 2d numpy array from those elements
          )
y_train = np.stack(y_train.values, axis = 0)

'''
#maybe can use this as example weights -- higher signal_to_noise means higher weight?
#probably gotta make sure to cap the weight though, otherwise training dominated by top signal_to_noise
signal_to_noise = train_df[signal_cols]  #not necessary anymore, although might still be useful -- try experimenting
'''


#np.array(train_df.drop(train_drop_cols, axis = 1).iloc[0].explode()).reshape(3,-1).astype('int')



In [ ]:
X_train

In [ ]:
y_train

In [ ]:
#even when filtered, it still might be useful to try sample weighting
sw = train_df['signal_to_noise'].values


import matplotlib.pyplot as plt

q = np.quantile(sw, np.linspace(0,1,21))
plt.plot(q, np.log1p(q + 5)/2)


sw = np.log1p(sw + 5)/2

## Define and Train Model

work on framework for save/loading weights

try integrating bpp as well

after this try graph network


Sequence model IO:

input: 3 sequences of same length (len == seq_length, which is either 68 or 91);
* input shape (n, 3, 107) or (n, 3, 130)

output: 5 sequences of seq_scored length;
* output shape (n, 5, 68) or (n, 5, 91)

In [ ]:
import tensorflow as tf
import tensorflow.keras.layers as layers
from tensorflow.keras.optimizers import Adam
#from tensorflow_addons.metrics import RSquare

#model should be highly generalizable, given limited public LB test data
#don't rely on public LB score too much
#input shape should be variable, since difference in input size for public vs private test set
#if taking in tabular data (ie for linear model), then input shape shouldn't matter


#lr = 0.01 seems pretty good for adam
TF_COMPILEPARAMS = {
    'optimizer': Adam(learning_rate = 0.01), #try different learning rates, optimizer hyperparameters
    'loss': score(use_tf = True),
    'metrics': ['mse']
}


def make_model():
    '''
    Creates a tensorflow keras sequence model
    '''
    #parameters passed into embedding layer
    EMBEDDING_PARAMS = {'input_dim': len(tokenizer.word_index) + 1,
                        'output_dim': 100, #play around with this number
                       }
    
    #should make this variable less prone to 
    shape = (3,None) #3 sequences of unknown length (should all be same length though)
    
    #create the model
    seq_inputs = tf.keras.Input(shape = shape) #shape (n, 3, seq_length)
    embed = layers.Embedding(**EMBEDDING_PARAMS)(seq_inputs) #(n, 3, seq_length, output_dim)
    
    #idea: 1 rnn per sequence, combine in single layer
    def rnn_layer(num_neurons):
        return layers.Bidirectional(layers.LSTM(num_neurons, return_sequences = True, dropout = 0.2)) #define the rnn type to use
    
    #create list of rnn layers, one for each sequence, then concatenate
    rnn_layers = []
    for i in range(embed.shape[1]): #loop thru sequences
        #one 2-layer bidirectional rnn per sequence
        r = rnn_layer(30)(embed[:,i])
        r = rnn_layer(30)(r)
        rnn_layers.append(r) #r is shape (n, seq_length, num_rnn_neurons * 2 (b/c bidirectional))
    
    x = layers.Concatenate()(rnn_layers) #concatenate the rnn layers for each sequence (n, seq_length, num_rnn_neurons * 2 * 3)
    x = layers.Dense(100, activation = 'relu')(x) #(n, seq_length, 100)
    x = layers.Dense(5, activation = 'linear')(x) #need output layer of 5 x seq_scored; shape (n, seq_length, 5)
    
    x = tf.transpose(x, (0,2,1)) #reshape prediction for submitting output & computing loss -- (n, 5, seq_length)
    x = x[:, :, :-39] #compact sequence of 107/130 into 68/91 by removing last 39 elements (n, 5, seq_scored)
    
    #this means one model works for both length inputs (really, any length)
    model = tf.keras.Model(inputs = seq_inputs, outputs = x)
    '''
    #lr = 0.01 seems pretty good for adam
    optimizer = Adam(learning_rate = 0.01) #try different learning rates, optimizer hyperparameters
    loss = score(use_tf = True)
    #loss = 'mse'
    '''
    model.compile(**TF_COMPILEPARAMS)
    
    
    return model

def make_model_bpp(seq_model):
    #right now this is temp code -- refactor when done experimenting
    '''
    maybe force seq_model to be fitted first
    seq_model is a fitted sequence model, as defined in make_model
    combines sequence model predictions with bpp features
    
    by bootstrapping off trained seq_model, this should verify whether scores are indeed improving after adding bpp features
    '''
    
    #get the sequence model first
    #seq_model = make_model()
    seq_input = seq_model.input
    seq_layers = seq_model.layers
    intercept_layer = 12 #the layer we'll start adding stuff to
    for l in seq_layers[:intercept_layer]: #seq_model should be fitted already, don't train first few layers again
        l.trainable = False
        
    #[print(i, l) for i,l in enumerate(seq_layers)]
    
    #work on fixing bpp_shape to accept variable length input
    bpp_shape = (107, 107) #bpp shape is (seq_length, seq_length)
    bpp_input = tf.keras.Input(shape = bpp_shape) #this is the bpp feature input
    
    #intercept sequence model after the 1st Dense layer
    x = seq_layers[12].output #shape (n, seq_length, 100)
    x = layers.Concatenate()([x, bpp_input]) #combine bpp data with sequence model output (n, seq_length, 100 + seq_length)
    x = layers.Dense(250, activation = 'relu')(x) #then train layer to incorporate bpp features (n, seq_length, 100)
    x = layers.Dense(100, activation = 'relu')(x) #then train layer to incorporate bpp features (n, seq_length, 100)

    
    #now plug this back into the original model (later layers will need to train again)
    for l in seq_layers[intercept_layer+1:]: #get later layers
        x = l(x) #apply later layers to x
    
    model = tf.keras.Model(inputs = [seq_input, bpp_input], outputs = x)
    model.compile(**TF_COMPILEPARAMS)

    return model

In [ ]:
#DO THE TRAINING
'''
work on callbacks (lr scheduling, logging, etc)
'''
from tensorflow.keras.callbacks import LearningRateScheduler, EarlyStopping
from sklearn.model_selection import train_test_split

#these are the additional parameters that go into keras model fit
TF_FITPARAMS = {'epochs': 150,
               'batch_size':100,
               'validation_batch_size':50}
fp = TF_FITPARAMS

def schedule_func(epoch, lr):
    '''
    function passed to LearningRateScheduler to determine learning rate at each epoch
    keeps lr constant until certain percent of epochs have elapsed, then exponentially decreases lr
    '''
    #percent = epoch / fp['epochs'] #get epoch progress
    if epoch < 50:
        return lr
    else:
        return lr * np.exp(-0.13)
    
callbacks = [
    LearningRateScheduler(schedule_func),
    EarlyStopping(monitor = 'val_loss', mode = 'min', min_delta = 5e-5, patience = 10)
]



#instantiate model
model = make_model()
model.summary()
print(X_train.shape, y_train.shape)




#split data into training and validation
#debate about test_size -- models without train_test_split may just be better on public LB due to more training data
idx_tr, idx_val = train_test_split(range(len(X_train)), test_size = 0.1) #split indices rather than X & y directly, so that we can also corroborate X with bpp (same indices)

X_tr, y_tr = X_train[idx_tr], y_train[idx_tr]
X_val, y_val = X_train[idx_val], y_train[idx_val]

#then do fitting
%time history = model.fit(X_tr, y_tr, validation_data = (X_val, y_val), callbacks = callbacks, **fp)

In [ ]:
tf.keras.utils.plot_model(model, show_shapes=True)

In [ ]:
#try out bpp model

model_bpp = make_model_bpp(model)
model_bpp.summary()

bpp_tr, bpp_val = bpp_train[idx_tr], bpp_train[idx_val]

%time history_bpp = model_bpp.fit([X_tr, bpp_tr], y_tr, validation_data = ([X_val, bpp_val], y_val), callbacks = callbacks, **fp)

In [ ]:
tf.keras.utils.plot_model(model_bpp, show_shapes = True)

In [ ]:
def save_model(model):
    '''
    WIP
    model is a fitted tensorflow keras model 
    '''
    
    pass

In [ ]:
import matplotlib.pyplot as plt
def plot_learning_curves(metric_dict):
    '''
    metric_dict is of similar form to tensorflow history.history object
    '''
    
    #plot loss and all metrics
    x_len = len(metric_dict['loss']) / 10 #length of plot proportional to epochs
    plt.figure(figsize = (x_len,13))

    all_metrics = set(metric_dict.keys()) - set(['lr']) #plot learning rate seperately

    #first plot loss and related metrics
    plt.subplot(2,1,1)
    for metric in all_metrics:
        metric_history = metric_dict[metric]
        plt.plot(metric_history, label = metric)
    plt.legend()

    #then plot learning rate
    plt.subplot(2,1,2)
    plt.plot(metric_dict['lr'], label = 'lr')
    plt.legend()

In [ ]:
#plot history of model

plot_learning_curves(history.history)
plt.title('Sequence Model')
plt.show()
plot_learning_curves(history_bpp.history)
plt.title('Sequence Model w/ BPP')

plt.show()

# Cross Validate

considering low amount of examples in public LB, good cross-val incredibly important!

looks like you should aim for top 10% on public LB (but not too aggressively push for top 3) if aiming to win on private LB

keep good logs of cross-validation

How to determine if **good public LB** == **good private LB**? longer sequence may mean worse performance, not generalizable

* ### Idea: Try "cutting" each sequence given in training set (only train on 90 out of 107 elements or so per sequence), train on that, and then predict public LB or even training set -- see how this affects score

alternatively, try bayesian optimization for hyperparameter tuning

In [ ]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
#wip

param_grid = {}
model_sk = KerasRegressor(build_fn = make_model) #make sk_learn wrapped model to use sklearn hyperparameter optimization


# disable when submitting, takes long


from sklearn.model_selection import cross_val_score, KFold, GroupKFold
from sklearn.metrics import make_scorer, mean_squared_error

#use groupkfold, where each group is single RNA seequence
groups = train_df['id'].values
cv = GroupKFold(n_splits = 5)
#cv = KFold(n_splits = 5)
#scorer = make_scorer(lambda y_true, y_pred: mean_squared_error(y_true, y_pred, squared = False))
scorer = make_scorer(score())


cv_scores = cross_val_score(model, X_train, y_train, cv = cv, groups = groups, scoring = scorer,
                            fit_params = {**fp})
print(cv_scores)
sns.boxplot(cv_scores)
f'{cv_scores.mean():.6f}', f'{cv_scores.std():.6f}'
#consider what fold class to use -- seems like same id should not be in train and val sets
#consider a groupkfold
#if one id is one row, however, can probably just use regular KFold - depends on format of train_df
#for now, doesn't seem like we need to do any scaling/normalization on entire train set, so we're good

**cv scores (mean, stdev):**
Need to redo scores because we initially trained on all data, not just good data (SN_filter == 1)

No SW Simple Dense NN: ('0.392408', '0.000757')


In [ ]:
#plot effect of tuning hyperparameter on score

In [ ]:
#log cross-val scores and params to a separate file

# Submit

we predict public and private test data seperately, to avoid numpy confusion

need to have all 107/130 (seq_length) rows in actual submission, despite training on 68 rows only

right now model only works for public data

In [ ]:
test_df = read_json('../input/stanford-covid-vaccine/test.json')

test_df

In [ ]:
#filter and tokenize
test_public = test_df[test_df['seq_length'] == 107]
test_public = tokenize_df(test_public, tokenizer)

test_public

In [ ]:
test_private = test_df[test_df['seq_length'] == 130]
test_private = tokenize_df(test_private, tokenizer)


test_private

In [ ]:
#create public test set data
X_test_public = (test_public.drop(drop_cols, axis = 1) #selects only relevant columns
                    .apply(lambda row: [e for e in row], axis = 1) #concatenates each element in row to list
                    .apply(lambda e : np.array(e)) #creates 2d numpy array from those elements
          )
X_test_public = np.stack(X_test_public.values, axis = 0) #shape (n,3,107)

#create private test set data
X_test_private = (test_private.drop(drop_cols, axis = 1) #selects only relevant columns
                    .apply(lambda row: [e for e in row], axis = 1) #concatenates each element in row to list
                    .apply(lambda e : np.array(e)) #creates 2d numpy array from those elements
          )
X_test_private = np.stack(X_test_private.values, axis = 0) #shape (n,3,130)

In [ ]:
def create_sub_df(test_df, predictions, length):
    '''
    this method creates submission dataframe from a df with data, predictions for that data, and sequence length
    '''
    sub_df = test_df.drop(tokenize_cols + ['index','seq_scored'], axis = 1) #first drop feature and irrelevant columns
    #create a seqpos column, which is a list that holds seqpos
    sub_df['seqpos'] = sub_df.apply(lambda row: list(range(row['seq_length'])), axis = 1)
    sub_df = unpack_df_lists(sub_df, 'seqpos') #now unroll
    sub_df['id_seqpos'] = sub_df.apply(lambda row: row['id'] + '_' + str(row['seqpos']), axis = 1)
    
    
    def pad_pred(p, i):
        '''
        p is the prediction
        i is final length
        fills unscored rows with 0s
        '''
        start = list(p) #first elements
        padding = [0] * (i - len(start)) #add padding to make up difference
        return start + padding

    pred_df = pd.DataFrame([[pad_pred(l, length) for l in e] for e in predictions], columns = target_cols) #here, each row holds a list in each column
    pred_df = unpack_df_lists(pred_df, target_cols) #then unroll those lists
    pred_df.index = sub_df.index
    pred_df = pred_df.reset_index()

    sub_df = sub_df.reset_index()
    sub_df = sub_df[['id_seqpos']]
    
    print(sub_df.shape, pred_df.shape)
    
    sub_df = sub_df.join(pred_df).set_index('index')

    
    return sub_df

In [ ]:
#do actual prediction
#test_pred_public = model.predict(X_test_public) #shape (n,5,68)
#test_pred_private = model.predict(X_test_private) #shape (n,5,91)

#try out model bpp
test_pred_public = model_bpp.predict([X_test_public, read_bpp(test_public['id'])]) #shape (n,5,68)

'''temp code to fill in private predictions with zeros'''
test_pred_private = np.zeros((X_test_private.shape[0], 5, 91))

#create submission dataframes
sub_public = create_sub_df(test_public, test_pred_public, 107)
sub_private = create_sub_df(test_private, test_pred_private, 130)

In [ ]:
#this temporary cell is for filling in private data with 0s
"""
test_private = test_df[test_df['seq_length'] == 130]

sub_private = test_private[['id', 'seq_length']]
sub_private['seqpos'] = sub_private.apply(lambda row: list(range(row['seq_length'])), axis = 1) #create seqpos column


test_pred_private = model.predict(X_)

for c in target_cols: #fill in targets with 0s
    sub_private[c] = sub_private.apply(lambda row: [0] * row['seq_length'], axis = 1)

sub_private = unpack_df_lists(sub_private, ['seqpos'] + target_cols) #unroll lists
sub_private['id_seqpos'] = sub_private.apply(lambda row: row["id"]+'_'+str(row["seqpos"]), axis = 1) #create id_seqpos column
sub_private = sub_private.drop(['id','seq_length','seqpos'], axis = 1)
sub_private = sub_private[['id_seqpos'] + target_cols] #rearrange column order
"""


In [ ]:
#concat public and private predictions into single submission dataframe

sub_df = pd.concat([sub_public, sub_private]).convert_dtypes()
sub_df

sns.pairplot(pd.DataFrame(y_train.reshape(-1,5), columns = target_cols))

#fig = plt.figure(figsize = (20,10))
#ax = plt.subplot(1,1,1)

sns.pairplot(sub_df)
#sub_df.plot(y='deg_Mg_pH10', ax = ax)

sns.heatmap((sub_df[sub_df['reactivity'] != 0]).corr(), annot = True)

#good -- seems like predicted targets do have significant correlation between each other, as seen in training data
#although might be higher correlation than reality? try regularizing/dropout

In [ ]:
sub_df.to_csv('submission.csv', index = False)

from sklearn.metrics import mean_absolute_error, mean_squared_error

a = model.predict(X_test_public).ravel()
b = model_bpp.predict([X_test_public, read_bpp(test_public['id'])]).ravel()

mean_absolute_error(a,b), np.sqrt(mean_squared_error(a,b)) #just check how big the difference in prediction is with and w/o bpp